In [ ]:
import gspread

def get_first_row_from_google_sheet():
    # Авторизация по сервисному аккаунту
    gc = gspread.service_account(filename="config_gkey.json")
    
    # Открываем таблицу и первый лист
    wks = gc.open("web_parsing_baza_knig").sheet1
    
    # Получаем первую строку (индексация начинается с 1!)
    first_row = wks.row_values(1)
    
    # Если нужно первое значение первой строки:
    first_value = wks.cell(1, 1).value
    
    return first_row

# Вызов функции
print(get_first_row_from_google_sheet())


In [ ]:
import gspread
import pandas as pd

def update_google_sheet(df: pd.DataFrame, mode: str = "append"):
    """
    Обновление Google Sheets из DataFrame
    
    mode:
      - "append"   → добавить в конец
      - "prepend"  → добавить в начало (после заголовков)
      - "overwrite"→ полностью перезаписать
    """
    # Заменяем NaN/None на пустые строки
    df = df.fillna("")

    # Авторизация
    gc = gspread.service_account(filename="config_gkey.json")
    wks = gc.open("web_parsing_baza_knig").sheet1

    # Преобразуем DataFrame в список списков
    rows = df.values.tolist()
    assert all(isinstance(r, list) for r in rows), "Данные должны быть списком списков"

    if mode == "append":
        # Добавляем в конец
        wks.append_rows(rows, value_input_option="RAW")

    elif mode == "prepend":
        n_rows, n_cols = df.shape

        # Вставляем пустые строки правильной ширины
        wks.insert_rows([[""] * n_cols] * n_rows, row=2)

        # Обновляем заголовки
        header = df.columns.tolist()
        wks.update("A1", [header])
        
        # Записываем DataFrame сразу после заголовков
        wks.update("A2", rows)

    elif mode == "overwrite":
        # Полностью перезаписываем таблицу (с заголовками)
        wks.clear()
        header = [df.columns.tolist()]
        wks.update("A1", header + rows)

    else:
        raise ValueError("mode должен быть 'append', 'prepend' или 'overwrite'")

data = [
    {"user_id": "user_400", "city": "Oslo", "category_name": "Books", "price": 100,
     "activation_data": "2025-09-05", "user_typ": "basic", "year": 2025, "month": 9},
    {"user_id": "user_401", "city": "Helsinki", "category_name": "Music", "price": 200,
     "activation_data": "2025-09-05", "user_typ": "premium", "year": 2025, "month": 9}
]

df = pd.DataFrame(data)

# 🔹 Добавить в начало (сдвигая старые данные вниз)
update_google_sheet(df, mode="prepend")